# CSL4020: Deep Learning | Project
__Music Style Transfer for MIDI data to produce Beethoven style.__  

---
___Yash Shrivastava, B21CS079___  
___Chaitanya Gaur, B21ES007___  
___Muneshwar Mansi Kailash, B21CS047___  


## Libraries

In [ ]:
!pip install miditok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 7.6 MB/s eta 0:00:00


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np
import pandas as pd
import pathlib
import zipfile
from miditok import REMI
from miditok.pytorch_data import DataCollator, DatasetMIDI
from miditok.utils import split_files_for_training
from torch.utils.data import DataLoader
from google.colab import files
from sklearn.preprocessing import LabelEncoder
from torch import LongTensor

## Data-Preprocessing
**IMPORTANT**  
DO NOT RUN ANY OF THE CELLS IN THIS SECTION.  
START FROM THE NEXT SECTION.

### Data Loading & Extraction

In [ ]:
uploaded = files.upload()
zip_filename = list(uploaded.keys())[0]

Saving maestro-v3.0.0-midi.zip to maestro-v3.0.0-midi.zip


In [ ]:
extract_dir = "midi_data"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

### Data Loader Creation

In [ ]:
data_info = pd.read_csv("midi_data/maestro-v3.0.0/maestro-v3.0.0.csv")
data_info.head()

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,Alban Berg,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160
1,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
2,Alban Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433
3,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588
4,Alexander Scriabin,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508


In [ ]:
le = LabelEncoder()
data_info['canonical_composer'] = le.fit_transform(data_info['canonical_composer'])
le.classes_

array(['Alban Berg', 'Alexander Scriabin', 'Antonio Soler',
       'Carl Maria von Weber', 'Charles Gounod / Franz Liszt',
       'Claude Debussy', 'César Franck', 'Domenico Scarlatti',
       'Edvard Grieg', 'Felix Mendelssohn',
       'Felix Mendelssohn / Sergei Rachmaninoff', 'Franz Liszt',
       'Franz Liszt / Camille Saint-Saëns',
       'Franz Liszt / Vladimir Horowitz', 'Franz Schubert',
       'Franz Schubert / Franz Liszt',
       'Franz Schubert / Leopold Godowsky',
       'Fritz Kreisler / Sergei Rachmaninoff', 'Frédéric Chopin',
       'George Enescu', 'George Frideric Handel',
       'Georges Bizet / Ferruccio Busoni',
       'Georges Bizet / Moritz Moszkowski',
       'Georges Bizet / Vladimir Horowitz',
       'Giuseppe Verdi / Franz Liszt', 'Henry Purcell', 'Isaac Albéniz',
       'Isaac Albéniz / Leopold Godowsky', 'Jean-Philippe Rameau',
       'Johann Christian Fischer / Wolfgang Amadeus Mozart',
       'Johann Pachelbel', 'Johann Sebastian Bach',
       'Johann Seb

In [ ]:
data_info.head()

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,0,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160
1,0,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
2,0,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433
3,1,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588
4,1,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508


In [ ]:
beethoven_class = np.where(le.classes_=='Ludwig van Beethoven')
beethoven_class

(array([40]),)

In [ ]:
data_info.drop(columns=['canonical_title', 'year', 'audio_filename', 'duration'], inplace=True)
data_info.head()

,canonical_composer,split,midi_filename
0,0,train,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...
1,0,train,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...
2,0,train,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...
3,1,train,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...
4,1,validation,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...


In [ ]:
data_info.to_csv('data_info.csv')

In [ ]:
data_info[data_info['canonical_composer']==40].groupby('split').nunique()

,Unnamed: 0,canonical_composer,midi_filename
split,,,
test,24,1,24
train,110,1,110
validation,12,1,12


In [ ]:
data_info.groupby('canonical_composer').nunique()

,Unnamed: 0,split,midi_filename
canonical_composer,,,
0,3,1,3
1,35,3,35
2,1,1,1
3,1,1,1
4,1,1,1
5,45,2,45
6,5,2,5
7,31,2,31
8,3,1,3


In [ ]:
data_info[data_info['canonical_composer']==18].groupby('split').nunique()

,Unnamed: 0,canonical_composer,midi_filename
split,,,
test,25,1,25
train,145,1,145
validation,31,1,31


In [ ]:
def get_file_paths(split, composer_id):
    data = data_info['midi_filename'][(data_info['split'] == split) & (data_info['canonical_composer']==composer_id)]
    data = 'midi_data/maestro-v3.0.0/' + data
    return data.to_list()

In [ ]:
train_file_paths_beethoven = get_file_paths('train', 40)
train_file_paths_beethoven += get_file_paths('test', 40)
test_file_paths_beethoven = get_file_paths('validation', 40)

In [ ]:
train_file_paths_chopin = get_file_paths('train', 18)
train_file_paths_chopin += get_file_paths('validation', 18)
test_file_paths_chopin = get_file_paths('test', 18)

Currently I am using default values of REMI tokenizer, if needed we can modify this later.

In [ ]:
tokenizer = REMI()

In [ ]:
train_file_paths_beethoven_pathlib = [pathlib.Path('/content/'+i) for i in train_file_paths_beethoven]
split_files_for_training(
    files_paths=train_file_paths_beethoven_pathlib,
    tokenizer=tokenizer,
    save_dir=pathlib.Path("beethoven_chunks"),
    max_seq_len=1024,
)

Splitting music files (beethoven_chunks): 100%|██████████| 134/134 [00:01<00:00, 99.77it/s] 


[PosixPath('beethoven_chunks/2006/MIDI-Unprocessed_11_R1_2006_01-06_ORIG_MID--AUDIO_11_R1_2006_02_Track02_wav_0.midi'),
 PosixPath('beethoven_chunks/2006/MIDI-Unprocessed_11_R1_2006_01-06_ORIG_MID--AUDIO_11_R1_2006_02_Track02_wav_1.midi'),
 PosixPath('beethoven_chunks/2006/MIDI-Unprocessed_11_R1_2006_01-06_ORIG_MID--AUDIO_11_R1_2006_02_Track02_wav_2.midi'),
 PosixPath('beethoven_chunks/2006/MIDI-Unprocessed_11_R1_2006_01-06_ORIG_MID--AUDIO_11_R1_2006_02_Track02_wav_3.midi'),
 PosixPath('beethoven_chunks/2006/MIDI-Unprocessed_11_R1_2006_01-06_ORIG_MID--AUDIO_11_R1_2006_02_Track02_wav_4.midi'),
 PosixPath('beethoven_chunks/2006/MIDI-Unprocessed_11_R1_2006_01-06_ORIG_MID--AUDIO_11_R1_2006_02_Track02_wav_5.midi'),
 PosixPath('beethoven_chunks/2006/MIDI-Unprocessed_11_R1_2006_01-06_ORIG_MID--AUDIO_11_R1_2006_02_Track02_wav_6.midi'),
 PosixPath('beethoven_chunks/2006/MIDI-Unprocessed_11_R1_2006_01-06_ORIG_MID--AUDIO_11_R1_2006_02_Track02_wav_7.midi'),
 PosixPath('beethoven_chunks/2006/MIDI-U

In [ ]:
train_file_paths_chopin_pathlib = [pathlib.Path('/content/'+i) for i in train_file_paths_chopin]
split_files_for_training(
    files_paths=train_file_paths_chopin_pathlib,
    tokenizer=tokenizer,
    save_dir=pathlib.Path("chopin_chunks"),
    max_seq_len=1024,
)

Splitting music files (chopin_chunks): 100%|██████████| 176/176 [00:01<00:00, 125.10it/s]


[PosixPath('chopin_chunks/2004/MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_02_Track02_wav_0.midi'),
 PosixPath('chopin_chunks/2004/MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_02_Track02_wav_1.midi'),
 PosixPath('chopin_chunks/2004/MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_02_Track02_wav_2.midi'),
 PosixPath('chopin_chunks/2004/MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_02_Track02_wav_3.midi'),
 PosixPath('chopin_chunks/2004/MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_02_Track02_wav_4.midi'),
 PosixPath('chopin_chunks/2004/MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_02_Track02_wav_5.midi'),
 PosixPath('chopin_chunks/2004/MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_02_Track02_wav_6.midi'),
 PosixPath('chopin_chunks/2004/MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_02_Track02_wav_7.midi'),
 PosixPath('chopin_chunks/2004/MIDI-Unprocessed_SMF_05_R

In [ ]:
def get_composer_id(score, tok_seq, file_path):
    file = file_path[25:]
    data = data_info[data_info['midi_filename']==file]
    data.reset_index(inplace=True)
    return LongTensor([data.loc[0,'canonical_composer']])

In [ ]:
def get_beethoven(score,tok_seq,file_path):
  return LongTensor([0])

def get_chopin(score,tok_seq,path):
  return LongTensor([1])

In [ ]:
train_dataset_beethoven = DatasetMIDI(
    files_paths=list(pathlib.Path("beethoven_chunks").glob("**/*.midi")),
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
    pre_tokenize=True,
    func_to_get_labels=get_beethoven
)
test_dataset_beethoven = DatasetMIDI(
    files_paths=test_file_paths_beethoven,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
    pre_tokenize=True,
    func_to_get_labels=get_beethoven
)
train_dataset_chopin = DatasetMIDI(
    files_paths=list(pathlib.Path("chopin_chunks").glob("**/*.midi")),
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
    pre_tokenize=True,
    func_to_get_labels=get_chopin
)
test_dataset_chopin = DatasetMIDI(
    files_paths=test_file_paths_chopin,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
    pre_tokenize=True,
    func_to_get_labels=get_chopin
)

Pre-tokenizing: 100%|██████████| 25/25 [00:02<00:00, 12.19it/s]


In [ ]:
collator = DataCollator(tokenizer.pad_token_id)
train_loader_beethoven = DataLoader(train_dataset_beethoven, batch_size=16, collate_fn=collator, shuffle=True)
train_loader_chopin = DataLoader(train_dataset_chopin, batch_size=16, collate_fn=collator, shuffle=True)
test_loader_beethoven = DataLoader(test_dataset_beethoven, batch_size=16, collate_fn=collator)
test_loader_chopin = DataLoader(test_dataset_chopin, batch_size=16, collate_fn=collator)

In [ ]:
len(train_loader_beethoven)

200

In [ ]:
torch.save(train_loader_beethoven, 'train_loader_beethoven.pt')
torch.save(test_loader_beethoven, 'test_loader_beethoven.pt')
torch.save(train_loader_chopin, 'train_loader_chopin.pt')
torch.save(test_loader_chopin, 'test_loader_chopin.pt')

## Implementation

**START FROM HERE**

In [ ]:
# extract_dir = "midi_data"
# os.makedirs(extract_dir, exist_ok=True)
# with zipfile.ZipFile("maestro-v3.0.0-midi.zip", 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)
# The commented code is kept in case you face errors,
# DO NOT RUN THIS COMMENTED CODE UNLESS VERY NECESSARY

data_info = pd.read_csv("data_info.csv")

def get_composer_id(score, tok_seq, file_path):
    file = file_path[25:]
    data = data_info[data_info['midi_filename']==file]
    data.reset_index(inplace=True)
    return LongTensor([data.loc[0,'canonical_composer']])

In [ ]:
train_loader = torch.load('train_loader.pt', weights_only=False)
val_loader = torch.load('val_loader.pt', weights_only=False)
test_loader = torch.load('test_loader.pt', weights_only=False)

***Important***:  
Always call the below seed setter before running a new session of work.

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

In [ ]:
for batch in train_loader_beethoven:
    sample = batch['input_ids']
    label = batch['labels']
    print(sample[0])
    print(len(sample[0]))
    break

tensor([  1,   4, 205,  ...,  53, 109, 129])
1024
